In [ ]:
import numpy as np
import scipy.special
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from distinctipy import distinctipy

from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import tukey_hsd
from IPython.display import set_matplotlib_formats

In [ ]:
# Figure 1 plots
df = pd.read_csv("Data/EdU_dropout.csv")
df = df.dropna()
df["Percent_EdU_positive"] = df["Percent_EdU_positive"].apply(lambda x: x * 100)

for genotype in ["R16D01", "R12D09"]:
    filtered_df = df.loc[(df["Line"] == genotype)]

    plt.subplots(figsize=(5, 4))

    x_ticks = filtered_df.Time_hr.unique()
    x_labels = filtered_df.Time_hr.unique()
    plt.vlines(
        x=x_ticks, ymin=-1, ymax=101, colors="grey", linestyle="dotted", zorder=1
    )

    # plot pointplot
    p = sns.lineplot(
        x="Time_hr",
        y="Percent_EdU_positive",
        data=filtered_df,
        color="black",
        err_style="bars",
        errorbar=("ci", 95),
    )

    sns.scatterplot(
        x="Time_hr",
        y="Percent_EdU_positive",
        data=filtered_df,
        color="black",
        marker="o",
    )

    plt.ylim(-0.5, 102)
    plt.xlim(1, 100)
    plt.xlabel("Time ALH (hour)", fontsize=16)
    plt.ylabel("% EdU+ LexA cells", fontsize=16)
    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=0)
    plt.tick_params(axis="both", which="major", labelsize=16)
    plt.minorticks_on()
    p.tick_params(axis="x", which="minor", bottom=False)
    sns.despine()
    plt.savefig(f"SVGs/{genotype}_EdU.svg")
    plt.show()

df1 = df.groupby(['Line','Time_hr']).size()
print(df1)

In [ ]:
# Figure 2 plots 
df = pd.read_csv("Data/Svp_counts.csv")
df = pd.melt(
    df,
    id_vars=["Condition", "Antibody", "Time", "Relative_lobe_side", "File_name"],
    value_vars=["DM1", "DM2", "DM3", "DM4", "DM5", "DM6", "DL1", "DL2"],
    var_name="Lineage",
    value_name="Positive_or_negative",
)
df = df.dropna()
df = df.sort_values("Time")
df["Time"] = df["Time"].astype(str) + " ALH"
lineage_order = ["DM1", "DM2", "DM3", "DM4", "DM5", "DM6", "DL1", "DL2"]
df.Lineage = pd.Categorical(df.Lineage, categories=lineage_order, ordered=True)
is_positive = np.zeros(df.index.size)
is_positive[df["Positive_or_negative"] == "+"] = 1
df.insert(0, "is_positive", is_positive)
df["is_positive"] = df["is_positive"].apply(lambda x: x * 100)

df.to_csv("example.csv")

colors = distinctipy.get_colors(
    len(df.Lineage.unique()), colorblind_type="Deuteranomaly"
)

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")

y_lable = "% of Svp+ Type 2 NBs"
x_lable = "Time ALH (hour)"
plt.subplots(figsize=(7, 3))

p = sns.barplot(
    data=df,
    x="Time",
    y="is_positive",
    hue="Lineage",
    errorbar=("ci", 95),
    errwidth=1,
    palette=colors,
)

plt.ylim(0, 100)
plt.ylabel(y_lable, fontsize=16)
plt.xlabel(x_lable, fontsize=16)
plt.xticks(rotation=0)
plt.tick_params(axis="both", which="major", labelsize=10)
plt.legend()
plt.minorticks_on()
p.tick_params(axis="x", which="minor", bottom=False)
sns.despine()
plt.savefig("SVGs/Svp_WT.svg")

plt.show()

df1 = df.groupby(['Lineage','Time']).size()
print(df1)

In [ ]:
# Supplemental Figure 1 - svp in situ
df = pd.read_csv("Data/Svp_counts_insitu.csv")
df = pd.melt(
    df,
    id_vars=["Condition", "Probe", "Time", "Relative_lobe_side", "File_name"],
    value_vars=["DM1", "DM2", "DM3", "DM4", "DM5", "DM6", "DL1", "DL2"],
    var_name="Lineage",
    value_name="Positive_or_negative",
)
df = df.dropna()
df = df.sort_values("Time")
df["Time"] = df["Time"].astype(str) + " ALH"
lineage_order = ["DM1", "DM2", "DM3", "DM4", "DM5", "DM6", "DL1", "DL2"]
df.Lineage = pd.Categorical(df.Lineage, categories=lineage_order, ordered=True)
is_positive = np.zeros(df.index.size)
is_positive[df["Positive_or_negative"] == "+"] = 1
df.insert(0, "is_positive", is_positive)
df["is_positive"] = df["is_positive"].apply(lambda x: x * 100)

df.to_csv("example.csv")

colors = distinctipy.get_colors(
    len(df.Lineage.unique()), colorblind_type="Deuteranomaly"
)

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")

y_lable = "% of mRNA Svp+ Type 2 NBs"
x_lable = "Time ALH (hour)"
plt.subplots(figsize=(7, 3))

p = sns.barplot(
    data=df,
    x="Time",
    y="is_positive",
    hue="Lineage",
    errorbar=("ci", 95),
    errwidth=1,
    palette=colors,
)

plt.ylim(0, 100)
plt.ylabel(y_lable, fontsize=16)
plt.xlabel(x_lable, fontsize=16)
plt.xticks(rotation=0)
plt.tick_params(axis="both", which="major", labelsize=10)
plt.legend()
plt.minorticks_on()
p.tick_params(axis="x", which="minor", bottom=False)
sns.despine()
plt.savefig("SVGs/Svp_mRNA_WT.svg")

plt.show()

df1 = df.groupby(['Lineage','Time']).size()
print(df1)

In [ ]:
# Supplemental Figure 2 plots and stats
df = pd.read_csv("Data/Svp_KO_larvae.csv")
df = pd.melt(df, id_vars=['Condition','Antibody','Time','% _positive_brain','File_name'],
            value_vars=['% _positive_Right_Lobe','%_positive_Left_Lobe'],
            var_name='Lobe', value_name='percent_positive')
df = df.dropna()
df['percent_positive'] = df['percent_positive'].apply(lambda x: x*100)
df["Condition"].replace(
{
    "Cas9_P2_control": "Cas9.P2 (control)",
    "Svp_KO_341527": "Svp-KO #1",
    "Svp_KO_341390": "Svp-KO #2",
},
inplace=True,
)
cat_order = ["Cas9.P2 (control)", "Svp-KO #1", "Svp-KO #2"]

colors = ["#00ff00", "#ff00ff", "#0080ff"]
    
for antibody in ['E93','Svp']:
    filtered_df = df.loc[(df['Antibody'] == antibody)]
    
    sns.set_context("talk")
    sns.axes_style()
    sns.set_theme(style='ticks')
    set_matplotlib_formats('retina') 

    plt.subplots(figsize=(4,3.5))

    x_var="Condition"
    y_var="percent_positive"

    y_label="% of T2NBs " + antibody

    # plot swarmplot
    p = sns.swarmplot(
        x=x_var, 
        y=y_var, 
        data=filtered_df, 
        hue=x_var, 
        order=cat_order,
        zorder=1,
        palette= colors,
        s=5
        )
        
    # plot boxplot
    sns.boxplot(
        x=x_var,
        y=y_var,
        data=filtered_df,
        showcaps=False,
        boxprops={"facecolor": "None"},
        showfliers=False,
        zorder=0,
        ax=p,
        order=cat_order,
        palette= colors
    )
    plt.ylim(-1.5,104)
    plt.xlabel(None)
    plt.ylabel(y_label, fontsize=16)
    plt.xticks(rotation=0)
    plt.tick_params(axis='both', which='major', labelsize=10)
    plt.legend([],[], frameon=False)
    plt.minorticks_on()
    plt.tick_params(axis='x', which='minor', bottom=False)
    plt.tick_params(axis='x', which='major', bottom=False)
    sns.despine()
    plt.savefig(f'SVGs/{antibody}_Svp_KO.svg')
    plt.show()

for antibody in ['E93','Svp']:
    filtered_df = df.loc[(df['Antibody'] == antibody)]

    print(f"\n{antibody} stats \n")

    title = antibody
    A = "Cas9.P2 (control)"
    B = "Svp-KO #1"
    C = "Svp-KO #2"
    Cat = "Condition"
    Var = "percent_positive"
    
    temp_df = filtered_df[[Cat, Var]]
    
    sample_A = temp_df.loc[temp_df[Cat] == A, Var]
    sample_B = temp_df.loc[temp_df[Cat] == B, Var]
    sample_C = temp_df.loc[temp_df[Cat] == C, Var]

    F, P = stats.f_oneway(sample_A, sample_B, sample_C)
    print(f"One-way ANOVA: F-statistic = {F} p-value =  {P}")
    result = scipy.stats.tukey_hsd(sample_A, sample_B, sample_C)
    print(title, result)

df1 = df.groupby(['Condition','Antibody']).size()
print(df1)

In [ ]:
# Figure 3 and Supplemental Figure 3 plots 
df = pd.read_csv("Data/Svp_cut_adult.csv")

df1 = df.groupby(['Genotype','Antibody']).size()
print(df1)

df = pd.melt(
    df,
    id_vars=["Genotype", "Antibody", "File"],
    value_vars=[
        "Percent Double+",
        "Percent Runt+ Marker-",
        "Percent Marker+ Runt-",
        "Percent Double-",
    ],
    var_name="marker",
    value_name="percent_positive",
)
df = df.dropna()
df["percent_positive"] = df["percent_positive"].apply(lambda x: x * 100)
cat_order = [
    "Percent Double+",
    "Percent Runt+ Marker-",
    "Percent Marker+ Runt-",
    "Percent Double-",
]

colors = distinctipy.get_colors(4)

for antibody in ["Cut", "Svp"]:
    filtered_df = df.loc[(df["Antibody"] == antibody)]

    agg_df = (
        filtered_df.groupby(["Genotype", "marker"])["percent_positive"].mean().unstack()
    )

    sns.set_context("talk")
    sns.axes_style()
    sns.set_theme(style="ticks")
    set_matplotlib_formats("retina")
    bottom = 0

    x_label = "Genotype"
    y_labael = "% of LexA cells"

    fig, ax = plt.subplots()

    for key, color in zip(cat_order, colors):
        mean = agg_df[key]
        ax.bar(agg_df.index, mean, label=key, bottom=bottom, color=color)
        bottom = bottom + mean

    plt.ylim(0, 101)
    plt.xlabel(x_label, fontsize=16)
    plt.ylabel(y_labael, fontsize=16)
    plt.tick_params(axis="y", which="major", labelsize=10)
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc="upper left")
    plt.minorticks_on()
    plt.tick_params(axis="x", which="minor", bottom=False)
    plt.tick_params(axis="x", which="major", bottom=False)
    sns.despine()
    plt.title(antibody)
    fig.set_size_inches(3, 2.5)
    plt.savefig(f"SVGs/{antibody}_adult_LexA.svg")
    plt.show()

In [ ]:
# Figure 4 plots and stats
df = pd.read_csv("Data/Svp_KO_R16D01.csv")
df = df.dropna()
df["Condition"].replace(
{
    "Cas9_P2_control": "Control",
    "Svp_CRISPR_VDRC#341527": "Svp-KO#1",
    "Svp_CRISPR_VDRC#341390": "Svp-KO#2",
},
inplace=True,
)

df.rename(columns={"Runt+/647+ LexA cells" : "cell_number"}, inplace=True)

cat_order = ["Control", "Svp-KO#1", "Svp-KO#2"]

colors = ["#0080ff", "#00ff00", "#ff00ff"]

x_var = "Condition"
y_var = "cell_number"
x_label = "Conditions"
y_labael = "P-FN cell number" '\n'  "(LexA+ and Runt+)"
title = "P-FN"

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")

plt.subplots(figsize=(6,4))

# plot swarmplot
p = sns.swarmplot(
    x=x_var,
    y=y_var,
    data=df,
    hue=x_var,
    order=cat_order,
    zorder=1,
    palette=colors
)

# plot boxplot
sns.boxplot(
    x=x_var,
    y=y_var,
    data=df,
    showcaps=False,
    boxprops={"facecolor": "None"},
    showfliers=False,
    zorder=0,
    ax=p,
    order=cat_order
)

plt.ylim(-0.5, 60.)
plt.ylabel(y_labael, fontsize=16)
plt.xlabel(None)
plt.xticks(rotation=0)
plt.tick_params(axis="y", which="major", labelsize=10)
plt.tick_params(axis='x', which='major', bottom=False)
plt.legend([], [], frameon=False)
plt.minorticks_on()
p.tick_params(axis="x", which="minor", bottom=False)
sns.despine()
plt.savefig('SVGs/PFN_Svp_KO.svg')
plt.show()

A = "Control"
B = "Svp-KO#1"
C = "Svp-KO#2"
Cat = "Condition"
Var = y_var

sample_A = df.loc[df[Cat] == A, Var]
sample_B = df.loc[df[Cat] == B, Var]
sample_C = df.loc[df[Cat] == C, Var]

F, P = stats.f_oneway(sample_A, sample_B, sample_C)

print(f"One-way ANOVA: F-statistic = {F} p-value =  {P} ")

result = scipy.stats.tukey_hsd(sample_A, sample_B, sample_C)
print(title, result)

df1 = df.groupby(['Condition']).size()
print(df1)

In [ ]:
# Figure 5 plots and stats
df = pd.read_csv("Data/Svp_KO_R12D09.csv")
df = df.dropna()
df["Condition"].replace(
    {
        "Cas9_P2_control": "Control",
        "Svp_KO_VDRC341527": "Svp-KO#1",
        "Svp_KO_VDRC341390": "Svp-KO#2",
    },
    inplace=True,
)
cat_order = ["Control", "Svp-KO#1", "Svp-KO#2"]

colors = ["#ff00ff", "#00ff00", "#0080ff"]

x_var = "Condition"
y_var = "Runt+ Cut+ LexA cells"
x_label = "Conditions"
y_labael = "P-EN cell number" "\n" "(Runt+ Cut+ LexA+)"
title = "P-EN"

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")
plt.subplots(figsize=(4, 4))

# plot swarmplot
p = sns.swarmplot(
    x=x_var, y=y_var, data=df, hue=x_var, order=cat_order, zorder=1, palette=colors
)

# plot boxplot
sns.boxplot(
    x=x_var,
    y=y_var,
    data=df,
    showcaps=False,
    boxprops={"facecolor": "None"},
    showfliers=False,
    zorder=0,
    ax=p,
    order=cat_order,
)

plt.ylim(-0.5, 350)
plt.ylabel(y_labael, fontsize=16)
plt.xlabel(None)
plt.xticks(rotation=0)
plt.tick_params(axis="y", which="major", labelsize=10)
plt.tick_params(axis="x", which="major", bottom=False)
plt.legend([], [], frameon=False)
plt.minorticks_on()
p.tick_params(axis="x", which="minor", bottom=False)
sns.despine()
plt.savefig("SVGs/PEN_Svp_KO.svg")
plt.show()

A = "Control"
B = "Svp-KO#1"
C = "Svp-KO#2"
Cat = "Condition"
Var = y_var

sample_A = df[df[Cat] == A][Var]
sample_B = df[df[Cat] == B][Var]
sample_C = df[df[Cat] == C][Var]

F, P = stats.f_oneway(sample_A, sample_B, sample_C)

print(f"One-way ANOVA: F-statistic = {F} p-value =  {P}")

result = scipy.stats.tukey_hsd(sample_A, sample_B, sample_C)
print(title, result)

df1 = df.groupby(['Condition']).size()
print(df1)

In [ ]:
# Figure 4 and 6 plots and stats - Neuropil counts
df = pd.read_csv("Data/Svp_KO_CX_neuropils.csv")
df = df.dropna()

df = pd.melt(df, id_vars=['Genotype','Condition','Neuropil_stain','File'],
            value_vars=['Protocerebral bridge','Fan-shaped body','Ellipsoid body','Noduli'],
            var_name='Neuropil', value_name='Positive_or_negative')

is_positive = np.zeros(df.index.size)
is_positive[df['Positive_or_negative'] == "+"] = 1
df.insert(0, 'is_positive', is_positive)
df['is_positive'] = df['is_positive'].apply(lambda x: x*100)

df['Condition'].replace(
{
    "Cas9_P2_control": "Cas9.P2 (control)",
    "Svp_CRISPR_VDRC#341527": "Svp KO #1",
    "Svp_CRISPR_VDRC#341390": "Svp KO #2",
},
inplace=True,
)

df['Neuropil'].replace(
{
    'Protocerebral bridge': 'PB',
    'Fan-shaped body': 'FB',
    'Ellipsoid body': 'EB',
    'Noduli': 'NO'
},
inplace=True,
)

df1 = df.groupby(['Condition','Genotype','Neuropil']).size()
print(df1)

cat_order = ["Cas9.P2 (control)", "Svp KO #1", "Svp KO #2"]
colors = ["#ff00ff",  "#00ff00", "#0080ff"]

for genotype in df.Genotype.unique():
    filtered_df = df.loc[(df['Genotype'] == genotype)]
    
    sns.set_context("talk")
    sns.axes_style()
    sns.set_theme(style="ticks")
    set_matplotlib_formats("retina")

    plt.subplots(figsize=(4,3.5))

    y_lable = "% brains with LexA" '\n' "targeting neuropil"
    x_lable = "Neuropil"

    p = sns.barplot(
        data = filtered_df,
        x = 'Neuropil',
        y = 'is_positive',
        hue_order = cat_order,
        hue = 'Condition',
        errorbar = ('ci',95),
        errwidth=1,
        palette= colors
    )

    plt.title(genotype)
    plt.ylim(0, 100)
    plt.ylabel(y_lable, fontsize=16)
    plt.xlabel(x_lable, fontsize=16)
    plt.xticks(rotation=0)
    plt.tick_params(axis="y", which="major", labelsize=16)
    plt.minorticks_on()
    plt.tick_params(axis="x", which="both", bottom=False)
    sns.despine()
    plt.savefig(f'SVGs/{genotype}_CX_neuropil.svg')
    plt.show()

    

In [ ]:
# Figure 4 and 5 plots and stats - EdU drop out
df = pd.read_csv("Data/EdU_SvpKO.csv")
df = df.dropna()
df["Percent_EdU_positive"] = df["Percent_EdU_positive"].apply(lambda x: x * 100)
colors = distinctipy.get_colors(len(df.Condition.unique()))

df["Condition"].replace(
    {"Cas9.P2_control": "Control", "Svp_KO_341527": "Svp-KO#1"},
    inplace=True,
)

df1 = df.groupby(['Condition','Line','Time_hr']).size()
print(df1)

for genotype in ["R16D01", "R12D09"]:
    filtered_df = df.loc[(df["Line"] == genotype)]

    plt.subplots(figsize=(5, 4))
    sns.set_context("talk")
    sns.axes_style()
    sns.set_theme(style="ticks")
    set_matplotlib_formats("retina")
    x_ticks = filtered_df.Time_hr.unique()
    x_labels = filtered_df.Time_hr.unique()
    plt.vlines(
        x=x_ticks, ymin=-1, ymax=101, colors="grey", linestyle="dotted", zorder=1
    )

    # plot pointplot
    p = sns.lineplot(
        x="Time_hr",
        y="Percent_EdU_positive",
        data=filtered_df,
        hue="Condition",
        err_style="bars",
        errorbar=("ci", 95),
        palette=colors,
    )

    sns.scatterplot(
        x="Time_hr",
        y="Percent_EdU_positive",
        data=filtered_df,
        hue="Condition",
        palette=colors,
        marker="o",
    )

    plt.title(genotype)
    plt.ylim(-1, 101)
    plt.xlim(0, 100)
    plt.xlabel("Time ALH (hour)", fontsize=16)
    plt.ylabel("Percent EdU+ LexA cells", fontsize=16)
    plt.xticks(rotation=45)
    plt.tick_params(axis="both", which="major", labelsize=16)
    plt.minorticks_on()
    p.tick_params(axis="x", which="minor", bottom=False)
    sns.despine()
    plt.savefig(f"SVGs/{genotype}_EdU_SvpKO.svg")
    plt.show()

In [ ]:
# Supplemental Figure 4 plots and stats - CX neuropil volumes
df = pd.read_csv("Data/Imaris_neuropil_reconstruction.csv")

df = df.dropna()

df = pd.melt(
    df,
    id_vars=[
        "Genotype",
        "Condition",
        "Stain",
        "PB_lexA_overlap",
        "FB_lexA_overlap",
        "EB_lexA_overlap",
        "NO_lexA_overlap",
        "%_PB_overlap",
        "%_FB_overlap",
        "%_EB_overlap",
        "%_NO_overlap",
    ],
    value_vars=["PB_volume", "FB_volume", "EB_volume", "NO_volume"],
    var_name="Neuropil",
    value_name="volumes",
)

df["Condition"].replace(
    {"Cas9_P2_control": "Control", "Svp_CRISPR_VDRC#341527": "Svp-KO#1"},
    inplace=True,
)

df["Neuropil"].replace(
    {"PB_volume": "PB", "FB_volume": "FB", "EB_volume": "EB", "NO_volume": "NO"},
    inplace=True,
)

cat_order = ["PB", "FB", "EB", "NO"]
colors = ["#ff00ff", "#00ff00"]

x_var = "Neuropil"
y_var = "volumes"
y_lable = "Volume of neuropil (um^3)"
x_lable = "Neuropils"

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")
plt.subplots(figsize=(6, 4))

# plot swarmplot
p = sns.swarmplot(
    x=x_var,
    y=y_var,
    data=df,
    hue="Condition",
    order=cat_order,
    zorder=1,
    dodge=True,
    palette=colors,
    legend=False,
)

# plot boxplot
sns.boxplot(
    x=x_var,
    y=y_var,
    data=df,
    showcaps=False,
    boxprops={"facecolor": "None"},
    showfliers=False,
    hue="Condition",
    zorder=0,
    ax=p,
    order=cat_order,
    palette=colors,
)

plt.ylim(0, 150000)
plt.xlabel(x_lable, fontsize=16)
plt.ylabel(y_lable, fontsize=16)
plt.xticks(rotation=0)
plt.tick_params(axis="both", which="major", labelsize=16)
plt.minorticks_on()
p.tick_params(axis="x", which="minor", bottom=False)
plt.tick_params(axis="x", which="major", bottom=False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc="upper left")
sns.despine()
plt.savefig("SVGs/neuropils_volume.svg")
plt.show()


for neuropil in np.unique(df["Neuropil"]):
    filtered_df = df.loc[(df["Neuropil"] == neuropil)]

    A = "Control"
    B = "Svp-KO#1"
    Cat = "Condition"
    Var = "volumes"

    sample_A = filtered_df[filtered_df[Cat] == A][Var]
    sample_B = filtered_df[filtered_df[Cat] == B][Var]

    # Welch's t-test
    T, P = stats.ttest_ind(sample_A, sample_B, equal_var=False)

    print(f"{neuropil} t-test: t-statistic = {T} p-value =  {P}")

df1 = df.groupby(['Condition','Neuropil']).size()
print(df1)

In [ ]:
# Supplemental Figure 4 plots and stats - P-EN neuropil volumes
df = pd.read_csv("Data/Imaris_neuropil_reconstruction.csv")

df = df.dropna()

df = pd.melt(df, id_vars=['Genotype','Condition','Stain',
'%_PB_overlap','%_FB_overlap','%_EB_overlap','%_NO_overlap','PB_volume','FB_volume','EB_volume','NO_volume'],
            value_vars=['PB_lexA_overlap','FB_lexA_overlap','EB_lexA_overlap','NO_lexA_overlap'],
            var_name='Neuropil', value_name='volumes')

df['Condition'].replace(
{
    "Cas9_P2_control": "Control",
    "Svp_CRISPR_VDRC#341527": "Svp-KO#1"
},
inplace=True,
)

df['Neuropil'].replace(
{
    'PB_lexA_overlap': 'PB',
    'FB_lexA_overlap': 'FB',
    'EB_lexA_overlap': 'EB',
    'NO_lexA_overlap': 'NO'
},
inplace=True,
)

cat_order = ['PB','FB','EB','NO']
colors = ["#ff00ff",  "#00ff00"]

x_var='Neuropil'
y_var='volumes'
y_lable = "Volume of P-EN targeting (um^3)"
x_lable = "Neuropils"

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")
plt.subplots(figsize=(6,4))

# plot swarmplot
p = sns.swarmplot(
    x=x_var, 
    y=y_var, 
    data=df, 
    hue='Condition', 
    order=cat_order,
    zorder=1,
    dodge=True,
    palette = colors,
    legend = False
    )
    
# plot boxplot
sns.boxplot(
    x=x_var,
    y=y_var,
    data=df,
    showcaps=False,
    boxprops={"facecolor": "None"},
    showfliers=False,
    hue='Condition', 
    zorder=0,
    ax=p,
    order=cat_order,
    palette = colors,
)

plt.ylim(0,20000)
plt.xlabel(x_lable, fontsize=16)
plt.ylabel(y_lable, fontsize=16)
plt.xticks(rotation=0)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.minorticks_on()
p.tick_params(axis='x', which='minor', bottom=False)
plt.tick_params(axis='x', which='major', bottom=False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
sns.despine()
plt.savefig('SVGs/P-EN_volume.svg')
plt.show()


for neuropil in np.unique(df['Neuropil']):
    filtered_df = df.loc[(df['Neuropil'] == neuropil)]

    A = "Control"
    B = "Svp-KO#1"
    Cat = "Condition"
    Var = 'volumes'
    
    sample_A = filtered_df[filtered_df[Cat] == A][Var]
    sample_B = filtered_df[filtered_df[Cat] == B][Var]
    
    # Welch's t-test
    T, P = stats.ttest_ind(sample_A, sample_B, equal_var=False)

    print(f"{neuropil} t-test: t-statistic = {T} p-value =  {P}")

df1 = df.groupby(['Condition','Neuropil']).size()
print(df1)

In [ ]:
# Supplemental Figure 4 plots and stats - Neuropil and LexA overlap
df = pd.read_csv("Data/Imaris_neuropil_reconstruction.csv")

df = df.dropna()
df['PB'] = (df['%_PB_overlap']).apply(lambda x: x*100)
df['FB'] = (df['%_FB_overlap']).apply(lambda x: x*100)
df['EB'] = (df['%_EB_overlap']).apply(lambda x: x*100)
df['NO'] = (df['%_NO_overlap']).apply(lambda x: x*100)

df = pd.melt(df, id_vars=['Genotype','Condition','Stain','PB_volume','PB_lexA_overlap',
'FB_volume','FB_lexA_overlap','EB_volume','EB_lexA_overlap','NO_volume','NO_lexA_overlap'],
            value_vars=['PB','FB','EB','NO'],
            var_name='Neuropil', value_name='percent_overlap')

df['Condition'].replace(
{
    "Cas9_P2_control": "Control",
    "Svp_CRISPR_VDRC#341527": "Svp-KO#1"
},
inplace=True,
)

cat_order = ['PB','FB','EB','NO']
colors = ["#ff00ff",  "#00ff00"]

x_var='Neuropil'
y_var='percent_overlap'
y_lable = "% overlap of" '\n' "neuropil with LexA"
x_lable = "Neuropils"

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")
plt.subplots(figsize=(6,4))

# plot swarmplot
p = sns.swarmplot(
    x=x_var, 
    y=y_var, 
    data=df, 
    hue='Condition', 
    order=cat_order,
    zorder=1,
    dodge=True,
    palette = colors,
    legend = False
    )
    
# plot boxplot
sns.boxplot(
    x=x_var,
    y=y_var,
    data=df,
    showcaps=False,
    boxprops={"facecolor": "None"},
    showfliers=False,
    hue='Condition', 
    zorder=0,
    ax=p,
    order=cat_order,
    palette = colors,
)

plt.ylim(0,100)
plt.xlabel(x_lable, fontsize=16)
plt.ylabel(y_lable, fontsize=16)
plt.xticks(rotation=0)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.minorticks_on()
plt.tick_params(axis='x', which='minor', bottom=False)
plt.tick_params(axis='x', which='major', bottom=False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
sns.despine()
plt.savefig('SVGs/neuropils_percent_overlap.svg')
plt.show()

for neuropil in np.unique(df['Neuropil']):
    filtered_df = df.loc[(df['Neuropil'] == neuropil)]

    A = "Control"
    B = "Svp-KO#1"
    Cat = "Condition"
    Var = 'percent_overlap'
    
    sample_A = filtered_df[filtered_df[Cat] == A][Var]
    sample_B = filtered_df[filtered_df[Cat] == B][Var]
    
    # Welch's t-test
    T, P = stats.ttest_ind(sample_A, sample_B, equal_var=False)

    print(f"{neuropil} t-test: t-statistic = {T} p-value =  {P}")

df1 = df.groupby(['Condition','Neuropil']).size()
print(df1)

In [ ]:
# Figure 7 - T2NB counts
df = pd.read_csv("Data/T2NB_decom.csv")
df = df.dropna()
df["Condition"].replace(
    {"Cas9.P2 control": "Control", "Svp_CRISPR_VDRC#341527": "Svp-KO"},
    inplace=True,
)

cat_order = ["24h APF", "48h APF", "1d Adult", "7d Adult"]

colors = ["#ff00ff", "#00ff00"]

x_var = "Time"
y_var = "Total_T2NB_number"
y_lable = "# of T2NBs per brain"
x_lable = "Time"

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")
plt.subplots(figsize=(6, 5))

# plot swarmplot
p = sns.swarmplot(
    x=x_var,
    y=y_var,
    data=df,
    hue="Condition",
    order=cat_order,
    zorder=1,
    dodge=True,
    palette=colors,
    legend=False,
)

# plot boxplot
sns.boxplot(
    x=x_var,
    y=y_var,
    data=df,
    showcaps=False,
    boxprops={"facecolor": "None"},
    showfliers=False,
    hue="Condition",
    zorder=0,
    ax=p,
    order=cat_order,
    palette=colors,
)

plt.ylim(-0.02, 15)
plt.xlabel(x_lable, fontsize=16)
plt.ylabel(y_lable, fontsize=16)
plt.xticks(rotation=0)
plt.tick_params(axis="both", which="major", labelsize=16)
plt.minorticks_on()
plt.tick_params(axis="x", which="minor", bottom=False)
plt.tick_params(axis="x", which="major", bottom=False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc="upper left")
sns.despine()
plt.savefig("SVGs/T2NB_decommisioning.svg")
plt.show()

for time in ["24h APF", "48h APF"]:
    filtered_df = df.loc[(df["Time"] == time)]

    A = "Control"
    B = "Svp-KO"
    Cat = "Condition"
    Var = "Total_T2NB_number"

    sample_A = filtered_df[filtered_df[Cat] == A][Var]
    sample_B = filtered_df[filtered_df[Cat] == B][Var]

    # t-test
    T, P = stats.ttest_ind(sample_A, sample_B, equal_var=False)

    print(f"{time} t-test: t-statistic = {T} p-value =  {P}")

df1 = df.groupby(['Condition','Time']).size()
print(df1)

In [ ]:
# Figure 7 - T2NB pH3+ counts 
df = pd.read_csv("Data/T2NB_H3.csv")
df['Condition'].replace(
{
    "Cas9.P2 control": "Control",
    "Svp_CRISPR_VDRC#341527": "Svp KO"
},
inplace=True,
)

cat_order = ["24h APF", "48h APF", "1d Adult", "7d Adult"]

colors = ["#00ff00"]

x_var='Time'
y_var='Total_T2NB_H3+'
y_lable = "# of pH3+ T2NBs per brain"
x_lable = "Time"

sns.set_context("talk")
sns.axes_style()
sns.set_theme(style="ticks")
set_matplotlib_formats("retina")
plt.subplots(figsize=(5,6))

p = sns.barplot(
    data = df,
    x=x_var, 
    y=y_var, 
    order=cat_order,
    palette = colors,
    errorbar="ci", capsize=0.2, errwidth=1.5, errcolor=".0"
    )

plt.ylim(0,2)
plt.xlabel(x_lable, fontsize=16)
plt.ylabel(y_lable, fontsize=16)
plt.xticks(rotation=0)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.minorticks_off()
plt.tick_params(axis='x', which='minor', bottom=False)
plt.tick_params(axis='x', which='major', bottom=False)
sns.despine()
plt.savefig('SVGs/T2NB_pH3.svg')
plt.show()

df1 = df.groupby(['Time']).size()
print(df1)